In [1]:
#import matplotlib as matplot
#import seaborn
#import bokeh
#import keras
#import scipy
import pandas as pd
import numpy as np
import networkx as nx
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
import plotly.graph_objs as go
from ncs1_import import ncs1_data=
from ncsr_import import ncsr_data
import re
import pathpy as pp
# import block

In [2]:
#ncs1 = ncs1_data()
ncsr = ncsr_data()
  # ncs1 holds all of the data from NCS 006693
  # Includes ncs1.dxdm (DS0002 DataFrame) and ncs1.survey (DS001 DataFrame)
  # Key Functions:
    # ncs1.search_for_description looks for a descriotion of a column name in ncs1.dxdm or ncs1.survey
  # Key Variables:
    # ncs1.dxdm (DS0002)
    # ncs1.survey (DS0001)
    # ncs1.tree (Tree including descriptions of survey and dxdm columns)
    # ncs1.root (root used in traversing ncs1.tree)

In [3]:
a = pd.DataFrame(columns = ['VarName', 'Description', 'Root_DF', 'Start', 'End', 'DataFrame', 'recursion_flag'])
for x in range(1, len(ncsr.root)):
    a = a.append(ncsr.get_value_from_string(ncsr.root.iloc[x,0])[ncsr.get_value_from_string(ncsr.root.iloc[x,0])['Description'].str.match(".*(first|1st|onset).*", False)])
a = a.reset_index(drop=True)


In [4]:
#a.loc[(a.loc[(a.loc[:,"Start"]< 3713), "Start"]>3369), :]
#https://www.kaggle.com/anand0427/network-graph-with-at-t-data-using-plotly

In [15]:
y = 444
varlist_444 = []
for x in a.iloc[:, 0]: 
    if ncsr.ncsr.loc[y, x] != ' ':
        if ncsr.ncsr.loc[y, x] != '.' and int(ncsr.ncsr.loc[y,x]) > 1 and int(ncsr.ncsr.loc[y,x]) < 100:
            if x not in ('MEDICATION', 'D37C2', 'D37C1', "SR10501", "SR128A01", "EM601", 'EM7_101', 'MR20', 'MR20A', "DA31B_101", "DE20_401", "DE20_901", "CH2_11",
            'CH6A1', 'T3A01', 'ID', 'PH5_03B', 'DM_EM7_1', 'DM_DA311'):
              varlist_444.append(x)

m18A - age of 1st excited/grouchy episode <br>
m18d1 - length of 1st excited/grouchy episode <br>
m18d2 - unit for m18d1 <br>
g27a_7 & g27a_7a = time & unit <BR>
T3A01 - 911

In [16]:
vals = list(ncsr.ncsr.loc[444, varlist_444])
vals = [int(i) for i in vals]
ordered_vals =[]
ordered_vars = []

for x in range(0, len(vals)):
    m = vals.index(min(vals))
    ordered_vals.append(vals[m])
    ordered_vars.append(varlist_444[m])
    del vals[m]
    del varlist_444[m]

In [20]:
node_info = {}
for x in range(0, len(ordered_vals)):
    node_info[ordered_vars[x]] = ordered_vals[x]

In [39]:
G = nx.Graph()
for x in ordered_vars:
    G.add_node(x)
nx.set_node_attributes(G, "AGE", node_info)
flag = 1
start = 0
for x in range(1, len(ordered_vars)):
    print(flag)
    if ordered_vals[x-1] != ordered_vals[x] and flag == 1:
        G.add_edges_from([(ordered_vars[x-1], ordered_vars[x])])
        start = x
    elif x + 1 == len(ordered_vals) and flag == 0:
        for y in range(start+1, x):
            G.add_edges_from([(ordered_vars[start], ordered_vars[y]), (ordered_vars[y], ordered_vars[x])])
        start = x
    elif x + 1 == len(ordered_vals) and flag == 1:
        G.add_edges_from([(ordered_vars[x-1], ordered_vars[x])])
        start = x
    elif flag == 0: 
        for y in range(start, x):
            print(str(y) + " - " + str(x))
            G.add_edges_from([(ordered_vars[start], ordered_vars[y]), (ordered_vars[y], ordered_vars[x])])
        flag = 1
        start = x
    else:
        flag = 0
        


1
1
0
1 - 3
2 - 3
1
0
3 - 5
4 - 5
1
0
5 - 7
6 - 7
1
1
1
1
1
0
11 - 13
12 - 13
1
0
13 - 15
14 - 15
1
0
15 - 17
16 - 17
1
0
17 - 19
18 - 19
1
0
19 - 21
20 - 21
1
0
21 - 23
22 - 23
1
0
23 - 25
24 - 25
1
0
25 - 27
26 - 27
1
0
27 - 29
28 - 29
1
0
29 - 31
30 - 31
1
1
0
32 - 34
33 - 34
1
0
34 - 36
35 - 36
1
0
36 - 38
37 - 38
1
0
38 - 40
39 - 40
1
1
0
41 - 43
42 - 43
1
0
43 - 45
44 - 45
1
0
45 - 47
46 - 47
1
1
0
48 - 50
49 - 50
1
0
50 - 52
51 - 52
1
0
52 - 54
53 - 54
1
1
0
55 - 57
56 - 57
1
1


In [106]:
#pos = nx.spring_layout(G, k=.4, iterations=100, dim=2, seed = 100)
pos = nx.kamada_kawai_layout(G)
for n, p in pos.items():
    G.nodes[n]['pos'] = p

edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=1,color='#888'),
    hoverinfo='none',
    mode='lines')

for edge in G.edges():
    x0, y0 = G.nodes[edge[0]]['pos']
    x1, y1 = G.nodes[edge[1]]['pos']
    edge_trace['x'] += tuple([x0, x1, None])
    edge_trace['y'] += tuple([y0, y1, None])



node_trace = go.Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='RdBu',
        #reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=10,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=0)))

for node in G.nodes():
    x, y = G.nodes[node]['pos']
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])

for node, adjacencies in enumerate(G.adjacency()):
    node_trace['marker']['color']+=tuple([len(adjacencies[1])])
    node_info = adjacencies[0] +' # of connections: '+str(len(adjacencies[1]))
    node_trace['text']+=tuple([node_info])


fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='Network for CASEID 445',
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="No. of connections",
                    showarrow=False,
                    xref="paper", yref="paper") ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))

iplot(fig)


In [78]:
nx.is_tree(G)

False